In [527]:
import cplex
from cplex.exceptions import CplexError
import sys
import pandas as pd
import numpy as np
import docplex

In [528]:
xls = pd.ExcelFile('logistic.xlsx')

vehicles = [1,2,3,4,5]                                # V
customers = [1,2,3,4,5,6,7,8,9]                       # C

num_vehicles = 2
num_customers = 2
num_locations = num_customers + 1                     
num_names = (num_locations ** 2) * num_vehicles

locations = [0] + customers[:num_customers]           # C': gồm các customers và Depot suất phát + Depot kết thúc

# quy ước: 
#    + (i,j) là cạnh (edge) thuộc tập E
#    + i,j là 2 customer liền kề trên 1 route

In [529]:
my_names = ["x"+str(i)+str(j) + str(k) for k in range(1,num_vehicles + 1) for i in range(0,num_locations) for j in range(0, num_locations) ]  

print(len(my_names))
print("Names ", my_names)

18
Names  ['x001', 'x011', 'x021', 'x101', 'x111', 'x121', 'x201', 'x211', 'x221', 'x002', 'x012', 'x022', 'x102', 'x112', 'x122', 'x202', 'x212', 'x222']


In [530]:
cost_matrix = pd.read_excel(xls, 'Cost Matrix')
my_obj = [int(cost_matrix.iat[i,j]) for i in range(0,num_locations ) for j in range(1,num_locations + 1)]*num_vehicles

print("Object ",my_obj)

Object  [0, 18, 30, 18, 0, 23, 30, 23, 0, 0, 18, 30, 18, 0, 23, 30, 23, 0]


In [531]:
capacity_matrix = pd.read_excel(xls, 'Capacity')
vehicles_capacity = [capacity_matrix.iat[i,1] for i in range(0, num_vehicles)]
vehicles_capacity

[100, 70]

In [532]:
demand_matrix = pd.read_excel(xls, 'Demand Matrix')
demand = {customers[i]: demand_matrix.iat[i,1] for i in range(0, num_customers)}
demand

{1: 32, 2: 45}

In [533]:
lower_bounds = [0]*num_names
upper_bounds = [1]*num_names

print("Lb ", lower_bounds)
print("Rb ", upper_bounds)

Lb  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Rb  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [567]:
# CONSTRAINTS_2: 
# # Một cạnh bất kỳ (i->j) chỉ tổn tại trên duy nhất 1 Vehicle. Nhưng i CHỈ LÀ customers, j là locations
contrains_2 = []
_i = 0
for i in customers[:num_customers]:
    for j in locations[:num_locations]:
        _i += 1
        con_ = [(1 if i != j and int(name[1]) == i and int(name[2]) == j else 0) for name in my_names]
        if len([c for c in con_  if c != 0]) > 0:
            contrains_2.append((con_,"E", 1, "C2" + str(_i)))
    
contrains_2

[([0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0], 'E', 1, 'C21'),
 ([0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0], 'E', 1, 'C23'),
 ([0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0], 'E', 1, 'C24'),
 ([0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0], 'E', 1, 'C25')]

In [568]:
# CONSTRAINTS_3: 
# # Lượng hàng chở đển Customer của 1 Vehicle phải nhỏ hơn bằng Capacity. Nhưng i CHỈ LÀ customers, j là locations
contrains_3 = []
_i = 0
for veh, cap in zip(vehicles[0:num_vehicles],vehicles_capacity[0:num_vehicles]): 
    _i += 1
    contrains_3.append(([int(demand[int(name[1])]) if (int(name[3]) == veh and int(name[1]) != 0) else 0 for name in my_names],"L", float(cap), "C3" + str(_i)))
contrains_3

[([0, 0, 0, 32, 32, 32, 45, 45, 45, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  'L',
  100.0,
  'C31'),
 ([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 32, 32, 32, 45, 45, 45],
  'L',
  70.0,
  'C32')]

In [569]:
# CONSTRAINTS_4: 
# # Với mỗi Vehicle, khi suất phát từ Depot, nó chỉ có thể đến duy nhất 1 Customer
contrains_4 = []
_i = 0
for k in vehicles[0:num_customers]:
    _i += 1
    contrains_4.append(([(1 if (int(name[1]) == 0 and int(name[3]) == k) else 0) for name in my_names],"E", 1, "C4" + str(_i)))
    
contrains_4

[([1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'E', 1, 'C41'),
 ([0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0], 'E', 1, 'C42')]

In [570]:
# CONSTRAINTS_5: 
# # Trên route của Vehicle k có cạnh (i->j) : tổng tất cả cạnh đầu 'ra' của i BẰNG tổng tất cả cạnh đầu 'vào' của j
contrains_5 = []
_i = 0
for k in vehicles[0:num_vehicles]:
    for p in customers[:num_customers]: # i
        for h in customers[:num_customers]: # j
            _i += 1
            foo = lambda k,p,h,n: 0 if int(n[3]) != k else 1 if int(n[2]) == p else -1 if int(n[1]) == h else 0                
            contrains_5.append(([foo(k, p, h, name) for name in my_names],"E", 0, "C5" + str(_i)))

contrains_5

[([0, 1, 0, -1, 1, -1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'E', 0, 'C51'),
 ([0, 1, 0, 0, 1, 0, -1, 1, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'E', 0, 'C52'),
 ([0, 0, 1, -1, -1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'E', 0, 'C53'),
 ([0, 0, 1, 0, 0, 1, -1, -1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'E', 0, 'C54'),
 ([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, -1, 1, -1, 0, 1, 0], 'E', 0, 'C55'),
 ([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, -1, 1, -1], 'E', 0, 'C56'),
 ([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, -1, -1, 1, 0, 0, 1], 'E', 0, 'C57'),
 ([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, -1, -1, 1], 'E', 0, 'C58')]

In [571]:
# CONSTRAINTS_6: 
# # Route của Vehicle k phải luôn trở về Depot (n+1), tức là luôn tồn tại điểm i trên route của k sao cho nó đi đến n+1
# # trong đó n+1 là Depot kết thúc
contrains_6 = []
_i = 0
for veh in vehicles[0:num_vehicles]: 
    _i += 1
    contrains_6.append(([(1 if (int(name[3]) == veh and int(name[2]) == 0 and int(name[1]) != 0) else 0) for name in my_names],"E", 1, "C6" + str(_i)))
    
contrains_6


[([0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'E', 1, 'C61'),
 ([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0], 'E', 1, 'C62')]

In [572]:
# CONSTRAINTS_X:
contrains_x = []
_i = 0

_i += 1
contrains_x.append(([(1 if int(name[1]) == int(name[2]) else 0) for name in my_names],"E", 0, "CX" + str(_i)))
    
contrains_x


[([1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1], 'E', 0, 'CX1')]

In [573]:
constraints_group = contrains_2 + contrains_6 + contrains_x

constraints_ = []
constraint_senses = []
rhs = []
constraint_names = []
for i in constraints_group:
    constraints_.append([my_names, i[0]])
    constraint_senses.append(i[1])
    rhs.append(i[2])
    constraint_names.append(i[3])
# print(constraints_)
# print(constraint_senses)
# print(rhs)


In [574]:
myProblem = None
try:
    myProblem = cplex.Cplex()     
except CplexError:
    print ("exc")
    
myProblem.variables.add(obj = my_obj,
                      lb = lower_bounds,
                      ub = upper_bounds,
                      names = my_names)

myProblem.objective.set_sense(myProblem.objective.sense.minimize)

for i in range(0, num_names):
    myProblem.variables.set_types(i,myProblem.variables.type.binary)

    
myProblem.linear_constraints.add(lin_expr = constraints_,
                                 senses = constraint_senses,
                                 rhs = rhs,
                                 names = constraint_names)

range(0, 7)

In [575]:
# And print the solutions
try:
    myProblem.solve()
    
    print(myProblem.get_stats())
    a=myProblem.solution.get_values()
    b=myProblem.variables.get_names()
    
    for i in range(len(a)):
        if a[i] != 0:
            print((b[i],a[i]))
except cplex.exceptions.errors.CplexSolverError:
    print("No solution")

Version identifier: 20.1.0.0 | 2020-11-11 | 9bedb6d68
CPXPARAM_Read_DataCheck                          1
Found incumbent of value 94.000000 after 0.00 sec. (0.00 ticks)
Tried aggregator 2 times.
MIP Presolve eliminated 4 rows and 15 columns.
Aggregator did 3 substitutions.
All rows and columns eliminated.
Presolve time = 0.00 sec. (0.01 ticks)

Root node processing (before b&c):
  Real time             =    0.00 sec. (0.02 ticks)
Parallel b&c, 8 threads:
  Real time             =    0.00 sec. (0.00 ticks)
  Sync time (average)   =    0.00 sec.
  Wait time (average)   =    0.00 sec.
                          ------------
Total (root+branch&cut) =    0.00 sec. (0.02 ticks)
Problem name         : 
Objective sense      : Minimize
Variables            :      18  [Binary: 18]
Objective nonzeros   :      12
Linear constraints   :       7  [Equal: 7]
  Nonzeros           :      18
  RHS nonzeros       :       6

Variables            : Min LB: 0.000000         Max UB: 1.000000       
Objective 